In [4]:
from pyspark.sql import SQLContext

sqlContext = SQLContext(sc)

df = sqlContext.read.load('file:///home/cloudera/Downloads/big-data-4/daily_weather.csv', 
                          format='com.databricks.spark.csv', 
                          header='true',inferSchema='true')

In [9]:
df.describe().printSchema()
df.toPandas().transpose()

root
 |-- summary: string (nullable = true)
 |-- number: string (nullable = true)
 |-- air_pressure_9am: string (nullable = true)
 |-- air_temp_9am: string (nullable = true)
 |-- avg_wind_direction_9am: string (nullable = true)
 |-- avg_wind_speed_9am: string (nullable = true)
 |-- max_wind_direction_9am: string (nullable = true)
 |-- max_wind_speed_9am: string (nullable = true)
 |-- rain_accumulation_9am: string (nullable = true)
 |-- rain_duration_9am: string (nullable = true)
 |-- relative_humidity_9am: string (nullable = true)
 |-- relative_humidity_3pm: string (nullable = true)



,0,1,2,3,4,5,6,7,8,9,...,1085,1086,1087,1088,1089,1090,1091,1092,1093,1094
number,0.000000,1.000000,2.000000,3.000000,4.000000,5.000000,6.000000,7.000000,8.000000,9.000000,...,1085.000000,1086.000000,1087.000000,1088.000000,1089.000000,1090.000000,1091.000000,1092.000000,1093.000000,1094.000000
air_pressure_9am,918.060000,917.347688,923.040000,920.502751,921.160000,915.300000,915.598868,918.070000,920.080000,915.010000,...,914.840000,921.260000,914.000000,912.900000,915.000000,918.900000,918.710000,916.600000,912.600000,921.530000
air_temp_9am,74.822000,71.403843,60.638000,70.138895,44.294000,78.404000,70.043304,51.710000,80.582000,47.498000,...,47.354000,52.646000,66.650000,71.870000,55.040000,63.104000,49.568000,71.096000,58.406000,77.702000
avg_wind_direction_9am,271.100000,101.935179,51.000000,198.832133,277.800000,182.800000,177.875407,242.400000,40.700000,163.100000,...,190.900000,261.900000,173.800000,129.200000,191.800000,192.900000,241.600000,189.300000,172.700000,97.100000
avg_wind_speed_9am,2.080354,2.443009,17.067852,4.337363,1.856660,9.932014,3.745587,2.527742,4.518619,4.943637,...,3.713320,2.035615,8.366156,1.431642,5.368656,3.869906,1.811921,3.064608,3.825167,3.265932
max_wind_direction_9am,295.400000,140.471548,63.700000,211.203341,136.500000,189.000000,186.606696,271.600000,63.000000,195.900000,...,204.400000,260.500000,181.000000,160.000000,220.900000,207.300000,227.400000,200.800000,189.100000,125.900000
max_wind_speed_9am,2.863283,3.533324,22.100967,5.190045,2.863283,10.983375,4.589632,3.646212,5.883152,6.576604,...,4.652835,3.042238,9.439887,2.057985,7.068730,5.212070,2.371156,3.892276,4.764682,4.451511
rain_accumulation_9am,0.000000,0.000000,0.000000,0.000000,8.900000,0.020000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
rain_duration_9am,0.000000,0.000000,20.000000,0.000000,14730.000000,170.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
relative_humidity_9am,42.420000,24.328697,8.900000,12.189102,92.410000,35.130000,10.657422,80.470000,29.580000,88.600000,...,92.300000,91.110000,30.920000,51.840000,73.550000,26.020000,90.350000,45.590000,64.840000,14.560000


In [11]:
df.describe(['air_temp_9am']).show()

+-------+------------------+
|summary|      air_temp_9am|
+-------+------------------+
|  count|              1090|
|   mean| 64.93300141287075|
| stddev|11.175514003175877|
|    min|36.752000000000685|
|    max| 98.90599999999992|
+-------+------------------+



In [12]:
df.count()

1095

In [13]:
removeAllDF = df.na.drop()

In [16]:
removeAllDF.describe(['air_temp_9am']).show()

+-------+------------------+
|summary|      air_temp_9am|
+-------+------------------+
|  count|              1064|
|   mean| 65.02260949558739|
| stddev|11.168033449415699|
|    min|36.752000000000685|
|    max| 98.90599999999992|
+-------+------------------+



In [17]:
from pyspark.sql.functions import avg
imputeDF = df

In [18]:
for x in imputeDF.columns:
    mean = removeAllDF.agg(avg(x)).first()[0]
    print(x, mean)
    imputeDF = imputeDF.na.fill(mean, [x])

number 545.0018796992481
air_pressure_9am 918.9031798641055
air_temp_9am 65.02260949558739
avg_wind_direction_9am 142.30675564934032
avg_wind_speed_9am 5.485793050713691
max_wind_direction_9am 148.48042413321312
max_wind_speed_9am 6.9997136588756925
rain_accumulation_9am 0.18202347650615522
rain_duration_9am 266.3936973996038
relative_humidity_9am 34.07743985327712
relative_humidity_3pm 35.14838093290537


In [20]:
df.describe(['air_temp_9am']).show()
imputeDF.describe(['air_temp_9am']).show()

+-------+------------------+
|summary|      air_temp_9am|
+-------+------------------+
|  count|              1090|
|   mean| 64.93300141287075|
| stddev|11.175514003175877|
|    min|36.752000000000685|
|    max| 98.90599999999992|
+-------+------------------+

+-------+------------------+
|summary|      air_temp_9am|
+-------+------------------+
|  count|              1095|
|   mean| 64.93341058219822|
| stddev|11.149948199920226|
|    min|36.752000000000685|
|    max| 98.90599999999992|
+-------+------------------+

